# Appending pandas Series

La función append nos permite concatenar objetos de tipo Series. Cuando hacemos esto, los índices de las distintas series concatenadas se mantienen. Si queremos resetear los índices podemos hacer uso de **reset_index(drop = True)**.

In [1]:
import pandas as pd
from glob import glob
#Importamos los datos
jan = pd.read_csv('sales-jan-2015.csv', index_col = 'Date', parse_dates = True)
feb = pd.read_csv('sales-feb-2015.csv', index_col = 'Date', parse_dates = True)
mar = pd.read_csv('sales-mar-2015.csv', index_col = 'Date', parse_dates = True)

#Extraemos de cada una de los series la columna Units
jan_units = jan.loc[:, 'Units']
feb_units = feb.loc[:, 'Units']
mar_units = mar.loc[:, 'Units']

#Concatenamos los valores 
quarter1 = jan_units.append(feb_units).append(mar_units)

#Extraemos info entre el 27 de enero y el 2 de febrero
print(quarter1.loc['jan 27, 2015': 'feb 2, 2015'])

Date
2015-01-27 07:11:55    18
2015-02-02 08:33:01     3
2015-02-02 20:54:49     9
Name: Units, dtype: int64


In [2]:
#Hacemos la suma de valores
print(quarter1.sum())

642


# Concatenating pandas Series along row axis

Además de **append()** podemos hacer uso de la función **concat()**. Esta función es mucho más versatil y potente que la función **append()**, ya que esta función nos permite concatenar tanto a nivel de filas como de columnas. El método **concat()**, dispone también del argumento **ignore_index = True**, que nos permite evitar que se mantengan los índices de las series originales.

In [4]:
#Hacemos uso de concat a nivel de fila
quarter1 = pd.concat([jan_units, feb_units, mar_units], axis = 'rows')

#Extraemos info entre el 27 de enero y el 2 de febrero
print(quarter1.loc['jan 27, 2015': 'feb 2, 2015'])

Date
2015-01-27 07:11:55    18
2015-02-02 08:33:01     3
2015-02-02 20:54:49     9
Name: Units, dtype: int64


# Appending DataFrames with ignore_index

El método **append()** también puede ser usado para objetos de tipo DataFrame, esta también tiene el argumento **ignore_index = True** que nos permite crear un nuevo index y no conservar los índices de los DataFrames originales.

In [10]:
#Cargamos los datos 
names_1881 = pd.read_csv('names1881.csv')
names_1981 = pd.read_csv('names1981.csv')

#Agregamos la columna year a cada uno de nuestros datasets
names_1881['year'] = 1881
names_1981['year'] = 1981

#Combinamos 
names = names_1881.append(names_1981, ignore_index = True)

#Vemos las dimensiones de nuestros datasets
print(names_1881.shape)
print(names_1981.shape)
print(names.shape)

(1935, 4)
(19455, 4)
(21390, 4)


In [11]:
#Mostramos todas las filas que tienen por nombre Morgan
names.query('name == "Morgan"')

,name,gender,count,year
1283,Morgan,M,23,1881
2096,Morgan,F,1769,1981
14390,Morgan,M,766,1981


# Concatenating pandas DataFrames along column axis

Si lo que deseamos es concatenar nuestros conjuntos de datos por columnas en lugar de por filas, debemos de hacer uso del método **concat()**, este método contiene el argumento axis que nos permite elegir el tipo de concatenación que deseamos realizar.

In [15]:
#Cargamos los datos
weather_max = pd.read_csv('weather_max.csv', index_col = 'Month')
weather_mean = pd.read_csv('weather_mean.csv', index_col = 'Month')

#Concatenamos por columna
weather_concat = pd.concat([weather_max, weather_mean], axis = 1)

#Vemos el resultado
weather_concat

,Max TemperatureF,Mean TemperatureF
Apr,89.0,53.100000
Aug,NaN,70.000000
Dec,NaN,34.935484
Feb,NaN,28.714286
Jan,68.0,32.354839
Jul,91.0,72.870968
Jun,NaN,70.133333
Mar,NaN,35.000000
May,NaN,62.612903
Nov,NaN,39.800000


# Reading multiple files to build a DataFrame

Existen situaciones en las cuales para construir nuestra dataframe definitivo tendremos que parsear varias conjuntos de datos y concatenarlos para crearnos nuestro DataFrame final. 

In [17]:
medal_type = ['gold', 'silver', 'bronze']
medals = []
for medal in medal_type:
    #Nos creamos el nombre del fichero del que queremos realizar la lectura
    file_name = "%s_top5.csv" % medal
    #Nos creamos la lista de columnas que deseamos parsear
    columns = ['Country', medal]
    #Leemos nuestro conjunto de datos
    df = pd.read_csv(file_name, index_col = 'Country', header = 0, names = columns)
    #Nos creamos una lista de dataframes
    medals.append(df)

medals = pd.concat(medals, axis = 'columns')
print(medals)

                  gold  silver  bronze
France             NaN   461.0   475.0
Germany          407.0     NaN   454.0
Italy            460.0   394.0     NaN
Soviet Union     838.0   627.0   584.0
United Kingdom   498.0   591.0   505.0
United States   2088.0  1195.0  1052.0


# Concatenating vertically to get MultiIndexed rows

Cuando apilamos múltiples DataFrames de forma vertical en un solo DataFrame, es deseable construir un MultiÍndice para indicar de donde provienen cada uno de los conjuntos de datos apilados. Esto se puede realizar haciendo uso del parámetro **keys** en **pd.concat()**, esto nos genera un índice jerárquico. 

In [3]:
medal_types = ['bronze', 'silver', 'gold']
medals = []
for medal in medal_types:
    file_name = "%s_top5.csv" % medal
    medal_df = pd.read_csv(file_name, index_col = 'Country')
    medals.append(medal_df)

medals = pd.concat(medals, keys = medal_types)
print(medals)

                        Total
       Country               
bronze United States   1052.0
       Soviet Union     584.0
       United Kingdom   505.0
       France           475.0
       Germany          454.0
silver United States   1195.0
       Soviet Union     627.0
       United Kingdom   591.0
       France           461.0
       Italy            394.0
gold   United States   2088.0
       Soviet Union     838.0
       United Kingdom   498.0
       Italy            460.0
       Germany          407.0


# Slicing MultiIndexed DataFrames

A continuación vamos a proceder a ordenar nuestro conjunto de datos medals y haremos uso de **pd.IndexSlice** para extraer información específica. 

In [8]:
#Ordenamos por el primer nivel
medals_sorted = medals.sort_index(level = 0)

#Vemos el número de medallas de bronce que gano Alemania
print(medals_sorted.loc[('bronze', 'Germany')])

Total    454.0
Name: (bronze, Germany), dtype: float64


In [9]:
#Mostramos la información solo respecto a las medallas de plata
print(medals_sorted.loc['silver'])

                 Total
Country               
United States   1195.0
Soviet Union     627.0
United Kingdom   591.0
France           461.0
Italy            394.0


In [12]:
#Nos creamos un IndexSlice
idx = pd.IndexSlice

#Mostramos todas las medallas ganadas por UK
print(medals_sorted.loc[idx[:, 'United Kingdom'], :])

                       Total
       Country              
bronze United Kingdom  505.0
gold   United Kingdom  498.0
silver United Kingdom  591.0


# Concatenating horizontally to get MultiIndexed columns

También es posible construir un DataFrame de forma jerárquica a partir de columnas. Para esto podemos hacer uso del argumento **axis** de **pd.concat()**.

In [22]:
#Hacemos la lectura de los datos
sales = []
for file in glob('feb-sales*.csv'):
    df = pd.read_csv(file, index_col = 'Date')
    sales.append(df)
    
#Procedemos a concatenar todo en un dataframe
february_sales = pd.concat(sales, keys = ['Software', 'Service', 'Hardware'], axis = 1)
print(february_sales.info())

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 2015-02-02 08:33:01 to 2015-02-26 08:58:51
Data columns (total 9 columns):
(Software, Company)    9 non-null object
(Software, Product)    9 non-null object
(Software, Units)      9 non-null float64
(Service, Company)     6 non-null object
(Service, Product)     6 non-null object
(Service, Units)       6 non-null float64
(Hardware, Company)    5 non-null object
(Hardware, Product)    5 non-null object
(Hardware, Units)      5 non-null float64
dtypes: float64(3), object(6)
memory usage: 1.6+ KB
None


In [28]:
#Nos creamos un elemento de tipo IndexSlice
idx = pd.IndexSlice

#Seleccionamos la información del 2 de febrero al 8 de febrero de la columna Company
print(february_sales.loc['2015-02-02':'2015-02-08', idx[:,'Company']])

                            Software Service         Hardware
                             Company Company          Company
2015-02-02 08:33:01            Hooli     NaN              NaN
2015-02-02 20:54:49              NaN     NaN        Mediacore
2015-02-03 14:14:18          Initech     NaN              NaN
2015-02-04 15:36:29        Streeplex     NaN              NaN
2015-02-04 21:52:45              NaN     NaN  Acme Coporation
2015-02-05 01:53:06  Acme Coporation     NaN              NaN
2015-02-05 22:05:03              NaN   Hooli              NaN
2015-02-07 22:58:10              NaN     NaN  Acme Coporation


# Concatenating DataFrames from a dict

In [38]:
#Cargamos los datos
jan = pd.read_csv('sales-jan-2015.csv')
feb = pd.read_csv('sales-feb-2015.csv')
mar = pd.read_csv('sales-mar-2015.csv')

#Nos creamos una lista de tuplas
month_list = [('january', jan), ('february', feb), ('march', mar)]

#Nos creamos un diccionario vacio
month_dict = {}

#Procedemos a crearnos nuestro conjunto de datos
for month_name, info in month_list:
    month_dict[month_name] = info.groupby('Company').sum()

sales = pd.concat(month_dict)
print(sales)

                          Units
         Company               
february Acme Coporation     34
         Hooli               30
         Initech             30
         Mediacore           45
         Streeplex           37
january  Acme Coporation     76
         Hooli               70
         Initech             37
         Mediacore           15
         Streeplex           50
march    Acme Coporation      5
         Hooli               37
         Initech             68
         Mediacore           68
         Streeplex           40


In [39]:
#Nos creamos un objeto de tipo IndexSlice
idx = pd.IndexSlice
#Obtemos todas las unidades vendidas por parte de Hooli para cada mes
print(sales.loc[idx[:, 'Mediacore'], :])

                    Units
         Company         
february Mediacore     45
january  Mediacore     15
march    Mediacore     68


# Concatenating DataFrames with inner join

A la hora de concatenar DataFrames tenemos la opción de hacer uso de DataFrames, para ello **pd.concat()** dispone del argumento **join**, este argumento puede tomar varios valores a continuación veremos el uso del valor **inner**. Cuando hacemos un **inner join** entre dos conjuntos de datos lo que hacemos es realizar un cruce entre índices y nos quedamos con las observaciones que tienen dicho índice común. En caso de que el número de las columnas fuese diferente, entonces en los valores que un determinado no tuviese valor, se pondría el valor de **NaN**.

In [44]:
#Cargamos los datos
medal_types = ['bronze', 'silver', 'gold']
medals = []
for medal in medal_types:
    file_name = "%s_top5.csv" % medal
    medal_df = pd.read_csv(file_name, index_col = 'Country')
    medals.append(medal_df)

#Hacemos el inner join 
print(pd.concat(medals, axis = 1, keys = medal_types, join = 'inner'))

                bronze  silver    gold
                 Total   Total   Total
Country                               
United States   1052.0  1195.0  2088.0
Soviet Union     584.0   627.0   838.0
United Kingdom   505.0   591.0   498.0


En este caso hemos realizado el inner join entre tres conjuntos de datos, en estos tres conjuntos solo los países: United States, Soviet Union y United Kingdom eran comunes por lo tanto tras realizar el inner solo nos quedamos con la información de estas tres observaciones.

# Resampling & concatenating DataFrames with inner join

A continuación vamos a realizar una comparación del producto interior bruto de los países de China y USA. Los datos de China empiezan en el año 1967 siendo estos datos anuales, mientras que los de USA se tratan de datos desde el año 1947 y están dados de forma trimestral.

In [76]:
#Cargamos los datos
china = pd.read_csv('gdp_china.csv', parse_dates = True)
china = pd.concat([pd.to_datetime(china.Year, format='%Y'), china.GDP], axis = 1)
china.set_index('Year', inplace = True)
china.columns = ['china']

usa = pd.read_csv('gdp_usa.csv', parse_dates = True, index_col = 'DATE')
usa.columns = ['usa']

#Procedemos a hacer un resample y calcular el porcentaje de cambio con un offset de 10 años
china_annual = china.resample('A').pct_change(10).dropna()
usa_annual = usa.resample('A').pct_change(10).dropna()

#Concatenamos los ambos conjuntos de datos
df_final = pd.concat([china_annual, usa_annual], axis = 1, join = 'inner')

#
print(df_final.resample('10A').last())

               china       usa
Year                          
1970-12-31  0.546128  0.980397
1980-12-31  1.072537  1.660540
1990-12-31  0.892820  1.088953
2000-12-31  2.357522  0.719980
2010-12-31  4.011081  0.455009
2020-12-31  3.789936  0.377506


/home/francisco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: 
.resample() is now a deferred operation
You called pct_change(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead
  # This is added back by InteractiveShellApp.init_path()
/home/francisco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: 
.resample() is now a deferred operation
You called pct_change(...) on this deferred object which materialized it into a dataframe
by implicitly taking the mean.  Use .resample(...).mean() instead
  if sys.path[0] == '':
